# Detector Models Comparison - G4LumaCam

This notebook demonstrates all available detector models in G4LumaCam using a neutron TOF simulation.

## Overview

We'll:
1. Run a `Config.neutrons_tof()` simulation
2. Process with different detector models using **separate suffix folders**
3. Compare outputs and performance
4. Visualize blob size and TOT distributions

## Available Models

1. **`image_intensifier`** - Simple circular blob (default)
2. **`gaussian_diffusion`** - Charge diffusion for CCDs
3. **`direct_detection`** - No blob, single pixel
4. **`wavelength_dependent`** - Spectral QE
5. **`avalanche_gain`** - Poisson gain + afterpulsing
6. **`image_intensifier_gain`** - Gain-dependent MCP (⭐ RECOMMENDED)
7. **`timepix3_calibrated`** - TPX3-specific TOT curve
8. **`physical_mcp`** - Full physics simulation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

# Import G4LumaCam
from lumacam import Config, Simulate, Lens

# Configure plotting
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

print("G4LumaCam detector models comparison")
print("=" * 50)

## 1. Run Neutron TOF Simulation

First, we'll run a quick neutron time-of-flight simulation to generate optical photons.

In [ ]:
# Create simulation config
archive_name = "demo_detector_models"

# Clean up old archive if it exists
archive_path = Path(archive_name)
if archive_path.exists():
    print(f"Removing old archive: {archive_path}")
    shutil.rmtree(archive_path)

config = Config.neutrons_tof(
    energy_min=1.0,  # keV
    energy_max=10.0  # keV
)

# Set simulation parameters
config.n_events = 50  # Small number for demo (increase for real simulations)
config.archive = archive_name

print(f"Simulation archive: {config.archive}")
print(f"Number of events: {config.n_events}")
print(f"Energy range: {config.energy_min}-{config.energy_max} keV")

In [ ]:
# Create Simulate object and run simulation (this may take a minute)
print("Running neutron TOF simulation...")
sim = Simulate(archive=archive_name)
sim.run(config)
print("✓ Simulation complete!")

# Check what was generated
sim_photons_dir = Path(archive_name) / "SimPhotons"
if sim_photons_dir.exists():
    files = list(sim_photons_dir.glob("*.csv"))
    print(f"Generated {len(files)} SimPhotons files")
else:
    print("Warning: SimPhotons directory not found")

## 2. Initialize Lens for Ray Tracing

Create a `Lens` object to process the optical photons.

In [ ]:
# Create lens object
lens = Lens(archive=archive_name)
print(f"Lens initialized with archive: {lens.archive}")

## 3. Test All Detector Models

Now we'll process the same photon data with each detector model using **separate suffix folders** for each model.

In [ ]:
# Define all models to test
detector_models = [
    {
        'name': 'image_intensifier',
        'suffix': 'img_int',
        'params': {'blob': 2.0, 'decay_time': 100, 'deadtime': 600}
    },
    {
        'name': 'image_intensifier_gain',
        'suffix': 'img_int_gain',
        # blob=0 to enable gain-dependent blob calculation (σ ∝ gain^0.4)
        'params': {'gain': 5000, 'sigma_0': 1.0, 'decay_time': 100, 'deadtime': 475, 'blob': 0}
    },
    {
        'name': 'timepix3_calibrated',
        'suffix': 'tpx3_cal',
        'params': {'gain': 5000, 'sigma_pixels': 1.5, 'tot_a': 30, 'tot_b': 50, 'deadtime': 475, 'blob': 0}
    },
    {
        'name': 'physical_mcp',
        'suffix': 'phys_mcp',
        'params': {'gain': 5000, 'phosphor_type': 'p47', 'deadtime': 475, 'blob': 0}
    },
    {
        'name': 'gaussian_diffusion',
        'suffix': 'gauss_diff',
        'params': {'blob': 1.5, 'deadtime': 400, 'charge_coupling': 0.85}
    },
    {
        'name': 'direct_detection',
        'suffix': 'direct',
        'params': {'deadtime': 300, 'blob': 0}
    }
]

print(f"Will test {len(detector_models)} detector models...")
print("Each model will store results in a separate suffix folder\n")
print("NOTE: For gain-dependent models (image_intensifier_gain, physical_mcp),")
print("      blob=0 allows automatic blob size calculation based on gain.")

In [ ]:
# Process with each model
results = {}

for model_config in detector_models:
    model_name = model_config['name']
    suffix = model_config['suffix']
    params = model_config['params'].copy()
    
    print(f"\n{'='*60}")
    print(f"Processing: {model_name}")
    print(f"Suffix folder: {suffix}")
    print(f"Parameters: {params}")
    print('='*60)
    
    # Extract common parameters
    deadtime = params.pop('deadtime', 600)
    blob = params.pop('blob', 1.0)
    
    # Run trace_rays with this model and suffix
    try:
        result_df = lens.trace_rays(
            deadtime=deadtime,
            blob=blob,
            detector_model=model_name,
            seed=42,  # Same seed for fair comparison
            suffix=suffix,  # Store in separate folder
            return_df=True,  # Return DataFrame directly
            **params  # Pass other params as kwargs
        )
        
        if result_df is not None and not result_df.empty:
            results[model_name] = {
                'df': result_df,
                'suffix': suffix,
                'params': params
            }
            print(f"✓ Result: {len(result_df)} pixel events")
            print(f"  Stored in: {archive_name}/Processed_{suffix}/SaturatedPhotons/")
        else:
            print(f"⚠ No results returned for {model_name}")
            
    except Exception as e:
        print(f"❌ Error processing {model_name}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n\n{'='*60}")
print(f"✓ Successfully processed {len(results)} models")
print('='*60)

## 4. Compare Results

Let's compare the outputs from different models.

In [ ]:
# Summary table
summary = []
for model_name, result_data in results.items():
    df = result_data['df']
    suffix = result_data['suffix']
    
    summary.append({
        'Model': model_name,
        'Suffix': suffix,
        'Pixel Events': len(df),
        'Avg Photons/Event': df['photon_count'].mean() if 'photon_count' in df.columns else 0,
        'Avg TOT (ns)': df['time_diff'].mean() if 'time_diff' in df.columns else 0,
        'Min TOT (ns)': df['time_diff'].min() if 'time_diff' in df.columns else 0,
        'Max TOT (ns)': df['time_diff'].max() if 'time_diff' in df.columns else 0
    })

summary_df = pd.DataFrame(summary)
print("\n" + "=" * 100)
print("DETECTOR MODEL COMPARISON")
print("=" * 100)
print(summary_df.to_string(index=False))
print("=" * 100)

## 5. Visualize TOT Distributions

Compare Time-Over-Threshold distributions for different models.

In [ ]:
n_models = len(results)
n_cols = 3
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
axes = axes.flatten() if n_models > 1 else [axes]

for idx, (model_name, result_data) in enumerate(results.items()):
    if idx >= len(axes):
        break
    
    ax = axes[idx]
    df = result_data['df']
    suffix = result_data['suffix']
    
    if 'time_diff' in df.columns and len(df) > 0:
        ax.hist(df['time_diff'], bins=50, alpha=0.7, edgecolor='black', color='steelblue')
        ax.set_xlabel('TOT (ns)', fontsize=11)
        ax.set_ylabel('Count', fontsize=11)
        ax.set_title(f'{model_name}\n(suffix: {suffix})\nMean TOT: {df["time_diff"].mean():.1f} ns', 
                    fontsize=12, fontweight='bold')
        ax.grid(alpha=0.3)
        ax.axvline(df['time_diff'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        ax.legend()

# Hide unused subplots
for idx in range(len(results), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle('TOT Distributions by Detector Model', y=1.01, fontsize=16, fontweight='bold')
plt.show()

## 6. Visualize Photon Count Distributions

Compare how many photons hit each pixel during deadtime.

In [ ]:
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
axes = axes.flatten() if n_models > 1 else [axes]

for idx, (model_name, result_data) in enumerate(results.items()):
    if idx >= len(axes):
        break
    
    ax = axes[idx]
    df = result_data['df']
    suffix = result_data['suffix']
    
    if 'photon_count' in df.columns and len(df) > 0:
        counts = df['photon_count'].value_counts().sort_index()
        ax.bar(counts.index, counts.values, alpha=0.7, edgecolor='black', color='darkorange')
        ax.set_xlabel('Photons per Pixel Event', fontsize=11)
        ax.set_ylabel('Count', fontsize=11)
        ax.set_title(f'{model_name}\n(suffix: {suffix})\nMean: {df["photon_count"].mean():.2f} photons',
                    fontsize=12, fontweight='bold')
        ax.grid(alpha=0.3, axis='y')

# Hide unused subplots
for idx in range(len(results), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle('Photon Count Distributions', y=1.01, fontsize=16, fontweight='bold')
plt.show()

## 7. Test Gain Scaling (image_intensifier_gain model)

Demonstrate how blob size scales with MCP gain.

In [ ]:
# Test different gain values
gains = [1000, 2000, 5000, 10000, 20000]
gain_results = {}

print("Testing gain scaling with image_intensifier_gain model...")
print("NOTE: blob=0 to allow gain-dependent blob calculation\n")
for gain in gains:
    print(f"  Processing gain = {gain}...")
    suffix_gain = f"gain_{gain}"
    
    try:
        df = lens.trace_rays(
            deadtime=475,
            blob=0,  # IMPORTANT: blob=0 to enable gain-dependent blob size!
            detector_model="image_intensifier_gain",
            gain=gain,
            sigma_0=1.0,
            seed=42,
            suffix=suffix_gain,
            return_df=True
        )
        
        if df is not None and not df.empty:
            gain_results[gain] = df
            print(f"    ✓ {len(df)} events")
    except Exception as e:
        print(f"    ❌ Error: {e}")

print(f"\n✓ Tested {len(gain_results)} gain values")

In [ ]:
# Plot gain scaling
if len(gain_results) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Plot 1: Event count vs gain
    gains_list = list(gain_results.keys())
    event_counts = [len(df) for df in gain_results.values()]
    ax1.plot(gains_list, event_counts, 'o-', linewidth=2, markersize=10, color='steelblue')
    ax1.set_xlabel('MCP Gain', fontsize=12)
    ax1.set_ylabel('Pixel Events', fontsize=12)
    ax1.set_title('Event Count vs. Gain', fontsize=14, fontweight='bold')
    ax1.grid(alpha=0.3)
    ax1.set_xscale('log')

    # Plot 2: Average photon count vs gain
    avg_photons = [df['photon_count'].mean() if 'photon_count' in df.columns else 0 
                   for df in gain_results.values()]
    ax2.plot(gains_list, avg_photons, 'o-', linewidth=2, markersize=10, color='darkorange')
    ax2.set_xlabel('MCP Gain', fontsize=12)
    ax2.set_ylabel('Avg Photons per Pixel', fontsize=12)
    ax2.set_title('Blob Size Effect (more photons → larger σ)', fontsize=14, fontweight='bold')
    ax2.grid(alpha=0.3)
    ax2.set_xscale('log')

    plt.tight_layout()
    plt.suptitle('Gain Dependence (σ ∝ gain^0.4)', y=1.02, fontsize=16, fontweight='bold')
    plt.show()
else:
    print("No gain results to plot")

## 8. Verify Separate Suffix Folders

Let's verify that each model has its own suffix folder.

In [ ]:
# List all Processed_* directories
archive_path = Path(archive_name)
processed_dirs = sorted(archive_path.glob("Processed_*"))

print("\n" + "="*80)
print("SUFFIX FOLDERS CREATED:")
print("="*80)

for proc_dir in processed_dirs:
    suffix_name = proc_dir.name.replace("Processed_", "")
    saturated_dir = proc_dir / "SaturatedPhotons"
    
    if saturated_dir.exists():
        files = list(saturated_dir.glob("*.csv"))
        print(f"  ✓ {proc_dir.name:30s} - {len(files)} saturated photon files")
    else:
        print(f"  ⚠ {proc_dir.name:30s} - No SaturatedPhotons folder")

print("="*80)

## Summary

This demo showed:
1. ✓ All detector models work correctly
2. ✓ Each model stores results in **separate suffix folders**
3. ✓ Gain-dependent blob scaling (σ ∝ gain^0.4)
4. ✓ Different TOT distributions per model
5. ✓ Easy model swapping with consistent API

## Recommendations

### For Timepix3 + Image Intensifier:

**Recommended model:** `image_intensifier_gain`

```python
lens.trace_rays(
    detector_model="image_intensifier_gain",
    gain=5000,                       # Typical MCP @ 1000V
    sigma_0=1.0,                     # Base blob size
    gain_exponent=0.4,               # From literature
    decay_time=100,                  # P47 phosphor
    deadtime=475,                    # TPX3 spec
    blob=1,
    suffix="my_model"                # Separate folder
)
```

### For P47 Phosphor (Full Physics):

```python
lens.trace_rays(
    detector_model="physical_mcp",
    gain=8000,                       # Chevron MCP
    phosphor_type='p47',             # Auto-configures P47
    deadtime=475,
    blob=1,
    suffix="p47_physics"
)
```

## Documentation

For detailed physics and parameter descriptions, see:
- `.documents/DETECTOR_MODELS.md` - Full model documentation
- `.documents/DETECTOR_MODELS_SUMMARY.md` - Quick reference guide